In [1]:
import os
import numpy as np
import tensorflow as tf

from ml.config.train import cfg as cfg_train
from ml.config.test import cfg as cfg_test
from ml.dataset import odgt2train, odgt2test
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder, TimingCallback

print("All packages imported!")

2024-01-10 11:06:51.474968: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


## Training

Example with static classifier.

### Config

In [2]:
# fill in your directory set up here
config_fp = '/home/jonathan/git/pdiot-ml/config/train/static.yaml'
cfg_train.merge_from_file(config_fp)
cfg_train.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_train.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/static/'   
print(cfg_train)

DATASET:
  LIST:
    train: train_static_pdiot-data.odgt
    val: val_static_pdiot-data.odgt
  num_classes: 5
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
MODEL:
  ARCH:
    LSTM:
      hidden_size: 0
      num_layers: 0
    MLP:
      dropout: 1.0
      hidden_size: 0
      num_layers: 0
  INPUT:
    format: summary
    sensor: all
    window_size: 50
TRAIN:
  DATA:
    batch_size: 128
    overlap_size: 0
  FN:
    config: config.yaml
    history: history.csv
    log: log.txt
    weight: weights.hdf5
  LEN:
    early_stop: 5
    num_epoch: 5
  LR:
    gamma: 0.1
    schedule: step
    step_size: 40
  OPTIM:
    lr: 0.01
    momentum: 0.9
    optim: adam
    weight_decay: 0.0005
  path: /home/jonathan/git/pdiot-ml/ckpt/static/


### Data

In [3]:
# experiment 
train_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.val)

train_X, train_y = odgt2train(train_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                            cfg_train.TRAIN.DATA.overlap_size)
val_X, val_y = odgt2train(val_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                      cfg_train.TRAIN.DATA.overlap_size)

### Optimization

In [4]:
model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg_train.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg_train.TRAIN.LEN.early_stop)

#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath=os.path.join(cfg_train.TRAIN.path, 'weights.hdf5'),
#    save_weights_only=True,
#    monitor='val_acc',
#    mode='max',
#    save_best_only=True)

#timing_callback = TimingCallback()

#history_callback = tf.keras.callbacks.CSVLogger(
#    os.path.join(cfg_train.TRAIN.path, 'history.csv'), 
#    separator=',', 
#    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg_train.TRAIN.LEN.num_epoch, 
          batch_size=cfg_train.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback])
#                     checkpoint_callback,
#                     timing_callback,
#                     history_callback])

Epoch 1/5


2024-01-10 11:06:59.865019: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


297/297 [==============================] - 2s 5ms/step - loss: 0.7542 - acc: 0.8382 - val_loss: 0.2920 - val_acc: 0.9845 - lr: 0.0100
Epoch 2/5
297/297 [==============================] - 1s 5ms/step - loss: 0.2048 - acc: 0.9818 - val_loss: 0.1666 - val_acc: 0.9895 - lr: 0.0100
Epoch 3/5
297/297 [==============================] - 2s 5ms/step - loss: 0.1363 - acc: 0.9843 - val_loss: 0.1327 - val_acc: 0.9900 - lr: 0.0100
Epoch 4/5
297/297 [==============================] - 2s 5ms/step - loss: 0.1135 - acc: 0.9846 - val_loss: 0.1199 - val_acc: 0.9925 - lr: 0.0100
Epoch 5/5
297/297 [==============================] - 2s 6ms/step - loss: 0.1031 - acc: 0.9847 - val_loss: 0.1131 - val_acc: 0.9925 - lr: 0.0100


## Testing

Example for task 1 with subject 29. 

### Config

In [5]:
# fill in your directory set up here
project_dp = '/home/jonathan/git/pdiot-ml/'
config_fp = os.path.join(project_dp, 'config/test/task_1.yaml')
cfg_test.merge_from_file(config_fp)
cfg_test.TEST.subject = 30
cfg_test.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_test.TEST.path = '/home/jonathan/git/pdiot-ml/ckpt/test/task_1'   
print(cfg_test)

DATASET:
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
  task: 1
MODEL:
  CONFIG:
    breath: 
    dynamic: config/train/task_1/dynamic.yaml
    motion: config/train/task_1/motion.yaml
    static: config/train/task_1/static.yaml
  INPUT:
    window_size: 50
TEST:
  DATA:
    batch_size: 128
    overlap_size: 0
  path: /home/jonathan/git/pdiot-ml/ckpt/test/task_1
  subject: 30


### Data

In [6]:
# experiment 
test_odgt_fp = os.path.join(cfg_test.DATASET.path, f'full_t{cfg_test.DATASET.task}_pdiot-data.odgt')
test_dict = odgt2test(test_odgt_fp, cfg_test.DATASET.task,
                                    cfg_test.TEST.subject, 
                                    cfg_test.MODEL.INPUT.window_size, 
                                    cfg_test.TEST.DATA.overlap_size)

### Optimization

In [ ]:
motion_cfg_fp = os.path.join(project_dp, cfg_test.MODEL.CONFIG.dynamic)
cfg_train.merge_from_file(motion_cfg_fp)
motion_model = ModelBuilder.build_classifier(cfg_train.MODEL, '', 5)


model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg_train.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg_train.TRAIN.LEN.early_stop)

#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath=os.path.join(cfg_train.TRAIN.path, 'weights.hdf5'),
#    save_weights_only=True,
#    monitor='val_acc',
#    mode='max',
#    save_best_only=True)

#timing_callback = TimingCallback()

#history_callback = tf.keras.callbacks.CSVLogger(
#    os.path.join(cfg_train.TRAIN.path, 'history.csv'), 
#    separator=',', 
#    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg_train.TRAIN.LEN.num_epoch, 
          batch_size=cfg_train.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback])
#                     checkpoint_callback,
#                     timing_callback,
#                     history_callback])

### Evaluation